In [2]:
%env OUTPUT_PATH=/Users/I559573/Downloads/D2V2.0/D2V_Datasets/ImageSamples
%env DATA_SOURCE=/Users/I559573/Downloads/D2V2.0/D2V_Datasets/ImageSamples/lgp_dataset

env: OUTPUT_PATH=/Users/I559573/Downloads/D2V2.0/D2V_Datasets/ImageSamples
env: DATA_SOURCE=/Users/I559573/Downloads/D2V2.0/D2V_Datasets/ImageSamples/lgp_dataset


In [20]:
# -*- coding: utf-8 -*-
"""
Training script to showcase the end-to-end training and evaluation script.
"""

import numpy as np
import pandas as pd
import datetime
import logging
import cv2
import joblib
import os

from os.path import exists
from joblib import load, dump
from os import makedirs
from os import environ
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


FORMAT = "%(asctime)s:%(name)s:%(levelname)s - %(message)s"
# Use filename="file.log" as a param to logging to log to a file
logging.basicConfig(format=FORMAT, level=logging.INFO)


class TrainSKInterface:
    def __init__(self) -> None:
        # Set the params for the training below
        self.image_pipeline = None
        self.dataset_all = None
        self.train, self.val, self.test = None, None, None
        self.target_classes = None
        self.dataset_name = "lgp_dataset"
        self.model_name = "classifier_pipeline.pkl"
        self.output_path = environ["OUTPUT_PATH"]
        self.file_name = environ["DATA_SOURCE"]


    def create_dataset_bin(self, img_folder):
        IMG_WIDTH=224
        IMG_HEIGHT=224
        img_data_array = []
        for file in os.listdir(img_folder):
            image_path = os.path.join(img_folder, file)
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image = np.array(image)
            image = image.astype('float32')
            image /= 255
            image = image.tobytes()
            img_data_array.append(image)
        return img_data_array 
    

    def read_dataset(self) -> None:
        """
        Reads the images file from path
        """
        
        path_img_ok = self.file_name + "/Images/OK/"
        path_img_ko = self.file_name + "/Images/NG/"
        
        logging.info(f"{path_img_ok}")
        logging.info(f"{path_img_ko}")
        
        img_dataset_ok_bin = self.create_dataset_bin(path_img_ok)
        img_dataset_ko_bin = self.create_dataset_bin(path_img_ko)

        df_img_dataset_ok = pd.DataFrame(columns = ['image','label'])
        df_img_dataset_ok['image'] = img_dataset_ok_bin
        df_img_dataset_ok['label'] = 0
        df_img_dataset_ko = pd.DataFrame(columns = ['image','label'])
        df_img_dataset_ko['image'] = img_dataset_ko_bin
        df_img_dataset_ko['label'] = 1

        self.dataset_all = pd.concat([df_img_dataset_ok,df_img_dataset_ko], ignore_index=True)
        self.dataset_all = self.dataset_all.sample(frac=1).reset_index(drop=True)
        self.target_classes = self.dataset_all["label"].unique()
        print(f"No. of training examples: {self.dataset_all.shape[0]}")
        print(f"Classes: {self.target_classes}")
        
        return None


    def split_dataset(self) -> None:
        """
        Split the dataset into train, validate and test

        Raises:
            Error: if dataset_train and dataset_test are not set
        """
        if self.dataset_all is None:
            raise Exception("Train or test data not set")

        #Change splitting proportions
        self.train, self.val = train_test_split(self.dataset_all, test_size=0.97, random_state=25)
        self.val, self.test = train_test_split(self.val, test_size=0.97, random_state=25)

        print(f"No. of training examples: {self.train.shape[0]}")
        print(f"No. of validation examples: {self.val.shape[0]}")
        print(f"No. of test examples: {self.test.shape[0]}")

        return None


    def convert_back(self, df):
        
        temp_arr = []
        for i in df['image'].values:
            a = np.frombuffer(i, dtype=np.float32)
            a = a.reshape(224,224,3)
            temp_arr.append(a)
            #print(a.shape)
            
        return temp_arr


    def prepare_model(self):
    
        base_model = tf.keras.applications.vgg16.VGG16(
            input_shape = (224, 224, 3), # Shape of our images
            include_top = False, # Leave out the last fully connected layer
            weights = 'imagenet'
        )

        for layer in base_model.layers:
            if(layer.name == 'block4_conv1'):
                break
            else:
                layer.trainable = False

        # Flatten the output layer to 1 dimension
        x = layers.Flatten()(base_model.output)

        # Add a fully connected layer with 512 hidden units and ReLU activation
        x = layers.Dense(512, activation='relu')(x)

        # Add a dropout rate of 0.5
        #x = layers.Dropout(0.5)(x) #To be uncommented

        # Add a final sigmoid layer with 1 node for classification output
        x = layers.Dense(1, activation='sigmoid')(x)

        self.image_pipeline = tf.keras.models.Model(base_model.input, x)

        self.image_pipeline.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-5),
                      loss = 'binary_crossentropy', 
                      metrics = ['accuracy']
                     )
        
        self.image_pipeline.summary()
        
        return None


    def train_model(self) -> None:
        """
        Train and save the model
        """
        
        img_train = self.convert_back(self.train)
        img_val = self.convert_back(self.val)
        
        print(len(img_train))
        print(len(img_val))
        print(img_train[0].shape)
        print(img_val[0].shape)

        self.image_pipeline.fit(
            x=np.array(img_train, np.float32), 
            y=np.array(list(map(int,self.train['label'])), np.float32), 
            validation_data = (np.array(img_val, np.float32), self.val['label'].values)
            #,steps_per_epoch = 100
            ,epochs = 1 #To be changed
        )

        return None


    def save_model(self) -> None:
        """
        Saves the model to the local path
        """
        
        logging.info(f"Writing tokenizer into {self.output_path}")
        if not exists(self.output_path):
            makedirs(self.output_path)
        # Save the Tokenizer and target classes to pickle file
        with open(f"{self.output_path}/{self.model_name}", "wb") as handle:
            dump([self.image_pipeline, self.target_classes], handle)

        return None


    def get_model(self) -> None:
        """
        Get the model if it is available locally
        """
        
        if exists(f"{self.output_path}/{self.model_name}"):
            logging.info(f"Loading classifier pipeline from {self.output_path}")
            with open(f"{self.output_path}/{self.model_name}", "rb") as handle:
                [self.image_pipeline, self.target_classes] = load(handle)
        else:
            logging.info(f"Model has not been trained yet!")

        return None


    def infer_model(self) -> str:
        """
        Perform an inference on the model that was trained
        """
        if self.image_pipeline is None:
            self.get_model()

        infer_data = np.array(self.convert_back(self.test), np.float32)
        logging.info(f"-----START INFERENCE-----")
        prediction = self.image_pipeline.predict(infer_data[0:1])
        predicted_label = "Anomalous" if prediction[0] > 0.5 else "Normal"
        logging.info(f"The input was predicted as '{predicted_label}'")
        logging.info(f"-----END INFERENCE-----")

        return predicted_label


    def run_workflow(self) -> None:
        """
        Run the training script with all the necessary steps
        """
        self.read_dataset()
        self.split_dataset()
            
        self.get_model()
        if self.image_pipeline is None:
            # Train the model if no model is available
            logging.info(f"Training classifier and saving it locally")
            self.prepare_model()
            self.train_model()
            self.save_model()

        self.infer_model()

        return None


if __name__ == "__main__":
    train_obj = TrainSKInterface()
    train_obj.run_workflow()


2022-02-18 18:09:19,940:root:INFO - /Users/I559573/Downloads/D2V2.0/D2V_Datasets/ImageSamples/lgp_dataset/Images/OK/
2022-02-18 18:09:19,941:root:INFO - /Users/I559573/Downloads/D2V2.0/D2V_Datasets/ImageSamples/lgp_dataset/Images/NG/
2022-02-18 18:09:23,661:root:INFO - Model has not been trained yet!
2022-02-18 18:09:23,665:root:INFO - Training classifier and saving it locally


No. of training examples: 822
Classes: [1 0]
No. of training examples: 24
No. of validation examples: 23
No. of test examples: 775
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     14758

2022-02-18 18:09:54,859:root:INFO - Writing tokenizer into /Users/I559573/Downloads/D2V2.0/D2V_Datasets/ImageSamples


INFO:tensorflow:Assets written to: ram://fa019346-9189-42d8-83a6-d96341136d40/assets


2022-02-18 18:10:01,643:tensorflow:INFO - Assets written to: ram://fa019346-9189-42d8-83a6-d96341136d40/assets
2022-02-18 18:10:04,428:root:INFO - -----START INFERENCE-----
2022-02-18 18:10:06,298:root:INFO - The input was predicted as 'Normal'
2022-02-18 18:10:06,299:root:INFO - -----END INFERENCE-----
